In [1]:
!pip install kiwipiepy

In [2]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import joblib

## 불용어 정의(상원)

In [3]:
stopword_n = pd.read_csv('keyword_n.csv', encoding= 'utf-8')
stopword_v = pd.read_csv('keyword_v2.csv', encoding= 'utf-8')


In [5]:
print(stopword_n)

     Keyword  Count
0         생각   4270
1         직업   4037
2         진로   4018
3         학생   3595
4         상담   3475
...      ...    ...
4512     관현악      1
4513      신입      1
4514      청음      1
4515    다운로드      1
4516     지난해      1

[4517 rows x 2 columns]


In [4]:
# Create stopwords list
stopwords = []

# Filter n: Keep only rows where 'Count' is 1000 or more
filtered_n = stopword_n[stopword_n['Count'] >= 1000]
stopwords.extend(filtered_n['Keyword'].tolist())

# Filter v: Keep only rows where 'Count' is 1
filtered_v = stopword_v[stopword_v['Count'] == 1]
stopwords.extend(filtered_v['Keyword'].tolist())

# Additional stopwords
additional_stopwords = [
    "안녕히", "안녕", "하세요", "크크크", "크크", "크크크크", "크크크크크", "이모티콘", "안녕하세요", 
    "키키", "키키키", "키키키키", "키키키키키", "키키키키키키", "키키키키키키키키", "키키키키키키키",
    "키키키키키키키키키키키키키키키키키", "키키키키키키키키키", "키키키키키키키키키키키키", "키키키키키키키키키키"
]
stopwords.extend(additional_stopwords)

print(stopwords)

['생각', '직업', '진로', '학생', '상담', '때', '관심', '학교', '관련', '일', '시간', '감사', '사람', '공부', '이야기', '선생', '학년', '친구', '분야', '검사', '정보', '고등학교', '말', '대학', '학과', '고민', '과목', '중요', '자신', '안녕', '부분', '도움', '진학', '꿈', '구부리', '비틀', '벗기', '내밀', '내보이', '큰일나', '집어던지', '뜯어고치', '안치', '미워하', '도리', '펴내', '으르', '매달', '지내보', '몰려다니', '헹구', '끄덕이', '야단맞', '날라다니', '찜하', '우러러보', '뛰어가', '쓰다듬', '쓰러지', '쫓겨나', '째지', '꾸물거리', '뒤집어쓰', '못생기', '굴리', '녹아들', '깨부수', '널브러지', '굴하', '찌그러지', '깔보', '헤엄치', '저버리', '까불거리', '끌어안', '청하', '기리', '늘어놓', '뛰쳐나오', '눌러두', '따라하', '공치', '담구', '쓸리', '껴들', '늘어서', '벼리', '따먹', '머뭇거리', '불거지', '중얼거리', '흘려버리', '정들', '격하', '돋우', '덧씌우', '떠나보내', '부풀리', '켕기', '아른거리', '장난삼', '썩히', '둘러대', '적시', '댕기', '정치', '가로막', '꼬시', '점찍', '몰라보', '꿰맞추', '끓어오르', '들춰내', '늦춰지', '몰아치', '더러워지', '타이르', '쩔쩔매', '빠져들', '내지르', '넘겨주', '애태우', '같아지', '갉아먹', '뒤척이', '틔우', '물밀', '꽃피', '추려내', '빗기', '짤리', '돋아나', '거머쥐', '달가워하', '동동거리', '알아먹', '뻥치', '돌아가시', '등지', '사들이', '데우', '알아맞추', '뛰놀', '쫓아내', '점하', '뛰어오', '차이', '오르내리', '들통나', '드높이', '갈아엎

## 모델링

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('train_kiwi.csv', encoding = 'utf-8')
test = pd.read_csv('test_kiwi.csv', encoding = 'utf-8')

In [4]:
from kiwipiepy import Kiwi

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

# 텍스트 전처리 함수
def preprocess_text_kiwi(text):
    result = kiwi.analyze(text)
    tokens = []
    for sentence in result:
        for word, pos, _, _ in sentence[0]:
            # 고유명사와 일반명사 품사를 추출
            if pos in ['NNG', 'NNP', 'VV']:
                tokens.append(word)
    return ' '.join(tokens)

# 텍스트 데이터 전처리
train['processed_conversation2'] = train['combined_conversation'].apply(preprocess_text_kiwi)
test['processed_conversation2'] = test['combined_conversation'].apply(preprocess_text_kiwi)

In [7]:
# 단어 길이가 1인 단어와 stopwords 제거 함수
def remove_single_char_and_stopwords(text, stopwords=None):
    if stopwords is None:
        stopwords = []

    words = text.split()
    filtered_words = [word for word in words if len(word) > 1 and word not in stopwords]
    return ' '.join(filtered_words)

# processed_conversation2 열에 함수 적용
train['processed_conversation2'] = train['processed_conversation2'].apply(lambda text: remove_single_char_and_stopwords(text, stopwords=stopwords))
test['processed_conversation2'] = test['processed_conversation2'].apply(lambda text: remove_single_char_and_stopwords(text, stopwords=stopwords))


In [11]:
train_test1 = pd.DataFrame({
    'label': train['priority1'],
    'text': train['processed_conversation2']
})

test_test1 = pd.DataFrame({
    'label': test['priority1'],
    'text': test['processed_conversation2']
})

In [12]:
train_test1.to_csv('train_test1.csv', index= False, encoding='utf-8-sig')
test_test1.to_csv('test_test1.csv', index= False, encoding='utf-8-sig')


In [17]:
# TF-IDF 벡터화 및 차원 축소(SVD) 파이프라인 설정
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=2)),
    ('svd', TruncatedSVD(n_components=100)),
])

# 학습 데이터 벡터화
X_train_tfidf = pipeline.fit_transform(train['processed_conversation2'])
y_train = train['priority1']

# 테스트 데이터 벡터화
X_test_tfidf = pipeline.transform(test['processed_conversation2'])
y_test = test['priority1']

In [18]:
# SVM 모델 학습 및 하이퍼파라미터 튜닝
svm_model = SVC(probability=True)
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf']
}
grid = GridSearchCV(svm_model, param_grid, refit=True, verbose=2)
grid.fit(X_train_tfidf, y_train)

# 최적의 하이퍼파라미터 출력
print("Best parameters found: ", grid.best_params_)

# 최적 모델 저장
best_model = grid.best_estimator_
with open('best_svm_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# 최적 모델 평가 및 확률 값 계산
y_pred = grid.predict(X_test_tfidf)
y_pred_proba = grid.predict_proba(X_test_tfidf)

print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 32 candidates, totalling 160 fits


d:\Anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   7.0s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   6.9s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   6.9s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   7.0s
[CV] END ......................C=0.1, gamma=1, kernel=linear; total time=   7.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.2s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=  11.2s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   7.0s
[CV] END ....................C=0.1, gamma=0.1, kernel=linear; total time=   7.0s
[CV] END ...................

NameError: name 'pickle' is not defined

In [19]:
# 최적 모델 평가 및 확률 값 계산
y_pred = grid.predict(X_test_tfidf)
y_pred_proba = grid.predict_proba(X_test_tfidf)

print(classification_report(y_test, y_pred))

                  precision    recall  f1-score   support

         IT관련전문직       0.45      0.65      0.53        43
       고급 운전 관련직       0.80      0.67      0.73         6
          공학 기술직       1.00      1.00      1.00         1
          공학 전문직       0.62      0.58      0.60        45
       교육관련 서비스직       0.59      0.59      0.59        22
     금융 및 경영 관련직       0.77      0.71      0.74        34
             기능직       0.50      0.14      0.22         7
기타 게임·오락·스포츠 관련직       1.00      0.38      0.55         8
       기타 특수 예술직       1.00      0.50      0.67         2
          기획서비스직       1.00      0.33      0.50         3
       농생명산업 관련직       0.00      0.00      0.00         2
         디자인 관련직       0.55      0.44      0.49        27
       매니지먼트 관련직       0.00      0.00      0.00         3
          무용 관련직       1.00      0.50      0.67         6
     미술 및 공예 관련직       0.56      0.56      0.56         9
   법률 및 사회활동 관련직       0.67      0.52      0.59        23
    보건의료 관련 서

d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 불용어(진섭)

In [14]:
# Create stopwords list
stopwords = []

# Filter n: Keep only rows where 'Count' is 1000 or more
filtered_n = stopword_n[stopword_n['Count'] >= 1000]
stopwords.extend(filtered_n['Keyword'].tolist())

# Filter v: Remove top 7 rows based on 'Count'
filtered_v = stopword_v.sort_values(by='Count', ascending=False).iloc[:7]
stopwords.extend(filtered_v['Keyword'].tolist())

# Additional stopwords
additional_stopwords = [
    "안녕히", "안녕", "하세요", "크크크", "크크", "크크크크", "크크크크크",
    "키키", "키키키", "키키키키", "키키키키키", "키키키키키키", "키키키키키키키키", "키키키키키키키",
    "키키키키키키키키키키키키키키키키키", "키키키키키키키키키", "키키키키키키키키키키키키", "키키키키키키키키키키"
]
stopwords.extend(additional_stopwords)

print(stopwords)

['생각', '직업', '진로', '학생', '상담', '때', '관심', '학교', '관련', '일', '시간', '감사', '사람', '공부', '이야기', '선생', '학년', '친구', '분야', '검사', '정보', '고등학교', '말', '대학', '학과', '고민', '과목', '중요', '자신', '안녕', '부분', '도움', '진학', '꿈', '대하', '좋아하', '모르', '나오', '위하', '가지', '정하', '안녕히', '안녕', '하세요', '크크크', '크크', '크크크크', '크크크크크', '키키', '키키키', '키키키키', '키키키키키', '키키키키키키', '키키키키키키키키', '키키키키키키키', '키키키키키키키키키키키키키키키키키', '키키키키키키키키키', '키키키키키키키키키키키키', '키키키키키키키키키키']


In [15]:
train = pd.read_csv('train_kiwi.csv', encoding = 'utf-8')
test = pd.read_csv('test_kiwi.csv', encoding = 'utf-8')

In [16]:
from kiwipiepy import Kiwi

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

# 텍스트 전처리 함수
def preprocess_text_kiwi(text):
    result = kiwi.analyze(text)
    tokens = []
    for sentence in result:
        for word, pos, _, _ in sentence[0]:
            # 고유명사와 일반명사 품사를 추출
            if pos in ['NNG', 'NNP', 'VV']:
                tokens.append(word)
    return ' '.join(tokens)

# 텍스트 데이터 전처리
train['processed_conversation2'] = train['combined_conversation'].apply(preprocess_text_kiwi)
test['processed_conversation2'] = test['combined_conversation'].apply(preprocess_text_kiwi)

In [17]:
# 단어 길이가 1인 단어와 stopwords 제거 함수
def remove_single_char_and_stopwords(text, stopwords=None):
    if stopwords is None:
        stopwords = []

    words = text.split()
    filtered_words = [word for word in words if len(word) > 1 and word not in stopwords]
    return ' '.join(filtered_words)

# processed_conversation2 열에 함수 적용
train['processed_conversation2'] = train['processed_conversation2'].apply(lambda text: remove_single_char_and_stopwords(text, stopwords=stopwords))
test['processed_conversation2'] = test['processed_conversation2'].apply(lambda text: remove_single_char_and_stopwords(text, stopwords=stopwords))


In [18]:
display(train.head(2))
display(test.head(2))


,student_idx,summary1,summary2,priority1,priority2,priority3,question1,question2,question3,question4,question5,question6,counsellor_idx,combined_conversation,processed_conversation,processed_conversation2
0,S-0155,학생은 독서와 글쓰기를 좋아하며 과학 과목을 좋아한다. 미래 희망 직업으로는 강아지...,"학생은 자연 친화력, 공간지각력, 신체 운동능력, 예술 시각의 적성이 모두 높은 수...",의료관련 전문직,이학 전문직,자연친화 관련직,4,3,4,4,5,3,T-0015,어서 오세요 [이모티콘] 방학인데 휴가는 다녀왔어요? 네! 어디로 다녀왔나요? 충주...,오 이모티콘 방학 휴가 다녀오 다녀오 충주 조 부모 댁 여행 학생 형제자매 되 여동...,이모티콘 방학 휴가 다녀오 다녀오 충주 부모 여행 형제자매 여동생 동생 동생 이모티...
1,S-0157,"진로상담을 처음 받아보면서 자기 자신이 좋아하는 것, 좋아하는 과목, 자기 자신을 ...",자신의 적성에 맞는 직업을 찾고 싶어 하여 상담사님과 함께 미래 유망 직업에 대해 ...,IT관련전문직,공학 전문직,공학 기술직,4,2,3,2,4,2,T-0045,안녕하세요? 학생은 초등학교 6학년인가요? 네 여름방학 생활은 어떤가요? 쉴 수 있...,안녕 학생 초등학교 학년 여름 방학 생활 쉬 있 오늘 진로 상담 하 있 처음 학년 ...,초등학교 여름 방학 생활 오늘 처음 학기 생활 국어 사회 수학 과학 즐기 활동 운동...


,student_idx,summary1,summary2,priority1,priority2,priority3,question1,question2,question3,question4,question5,question6,counsellor_idx,student167,combined_conversation,processed_conversation,processed_conversation2
0,S-0259,직업 선호도 검사를 받아본 적이 있으며 취미는 운동이다.\n운동과 게임을 좋아하지만...,지난 상담 시간에 체육 교사에 대하여 대화를 나누었는데 여전히 체육 교사에는 관심이...,운동 관련직,매니지먼트 관련직,안전 관련직,4,4,2,2,2,2,T-0028,NaN,안녕하세요? 안녕하세요 카카오톡 상담사입니다 지금 상담 가능한가요? 네. 비가 너무...,안녕 안녕 카카오톡 상담사 상담 가능 비 오 날씨 불구하 진로 탐색 위하 노력 모습...,카카오톡 상담사 가능 날씨 불구하 탐색 노력 모습 보이 지금 여름 방학 기간 방학 ...
1,S-0268,초등학교 6학년으로 재학 중에 있으며 아직 진로에 대해서 확실하게 결정하지는 못하였...,외국 고등학교 진학에 관심을 두고 있다. 진로를 결정하는 데 아직 어려움이 있고 이...,법률 및 사회활동 관련직,언어 관련 전문직,인문 및 사회과학 관련직,2,2,4,3,3,3,T-0031,NaN,안녕하세요~ 초등학교 6학년이네요~ 네 안녕하세요! 진로상담 학교에서 안내받은 거예...,안녕하세요 초등학교 학년 안녕 진로 상담 학교 안내 엄마 학부모회 네트워크 보 오 ...,안녕하세요 초등학교 안내 엄마 학부모회 네트워크 법조인 동기 바라보 바라 소아과 의...


In [19]:
train_test2 = pd.DataFrame({
    'label': train['priority1'],
    'text': train['processed_conversation2']
})

test_test2 = pd.DataFrame({
    'label': test['priority1'],
    'text': test['processed_conversation2']
})

train_test2.to_csv('train_test2.csv', index= False, encoding='utf-8-sig')
test_test2.to_csv('test_test2.csv', index= False, encoding='utf-8-sig')

## test3(stopwords 정의한 것만 사용)

In [ ]:
import pandas as pd

train = pd.read_csv('train_kiwi.csv', encoding = 'utf-8')
test = pd.read_csv('test_kiwi.csv', encoding = 'utf-8')

In [ ]:
from kiwipiepy import Kiwi

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

# 텍스트 전처리 함수
def preprocess_text_kiwi(text):
    result = kiwi.analyze(text)
    tokens = []
    for sentence in result:
        for word, pos, _, _ in sentence[0]:
            # 고유명사와 일반명사 품사를 추출
            if pos in ['NNG', 'NNP', 'VV']:
                tokens.append(word)
    return ' '.join(tokens)

# 텍스트 데이터 전처리
train['processed_conversation2'] = train['combined_conversation'].apply(preprocess_text_kiwi)
test['processed_conversation2'] = test['combined_conversation'].apply(preprocess_text_kiwi)

In [5]:
stopwords = [
    "안녕히", "안녕", "하세요", "크크크", "크크", "크크크크", "크크크크크", "이모티콘", "안녕하세요", 
    "키키", "키키키", "키키키키", "키키키키키", "키키키키키키", "키키키키키키키키", "키키키키키키키",
    "키키키키키키키키키키키키키키키키키", "키키키키키키키키키", "키키키키키키키키키키키키", "키키키키키키키키키키"
]

# 불용어 제거 함수
def remove_stopwords(text, stopwords):
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 불용어 제거 적용
train['processed_conversation2'] = train['processed_conversation2'].apply(lambda x: remove_stopwords(x, stopwords))
test['processed_conversation2'] = test['processed_conversation2'].apply(lambda x: remove_stopwords(x, stopwords))


In [6]:
train_test3 = pd.DataFrame({
    'label': train['priority1'],
    'text': train['processed_conversation2']
})

test_test3 = pd.DataFrame({
    'label': test['priority1'],
    'text': test['processed_conversation2']
})

from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# train_test1 데이터프레임의 'label' 열 레이블 인코딩
train_test3['label'] = label_encoder.fit_transform(train_test3['label'])

# test_test1 데이터프레임의 'label' 열도 같은 인코더로 레이블 인코딩 (필요시)
test_test3['label'] = label_encoder.transform(test_test3['label'])


In [9]:
train_test3.to_csv('train_test3.csv', index= False, encoding='utf-8-sig')
test_test3.to_csv('test_test3.csv', index= False, encoding='utf-8-sig')